In [ ]:
import obspy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# GEOL 593: Seismology and Earth Structure

## Lab assignment 4: Travel time inversion for 1D mantle structure

In class, we discussed the behavior of seismic rays, including how we can model ray paths for one dimensional (1-D) velocity profiles, in which velocity only varies with depth. For models in which velocity increases smoothly with depth, we saw that the travel times of body waves (e.g., P and S) smoothly increase as a function of distance, and the slope of the travel time curve gets progressively shallower with distance as rays turn deeper and deeper. 

In this assignment, your goal is to construct a travel time curve, $T(X)$, from seismic data, and use the curve to invert for the average 1-D velocity profile in Earth's mantle. To do this, you will download seismograms from a recent earthquake recorded at stations from the Global Seismograph Network, and pick P-wave travel times to constuct the $T(X)$ curve. Then, using the classic *Herglotz-Weichert* formula, you will invert the travel time curve.

Let's get started.

### Download data from a single earthquake

The analysis in this lab will be based off data from a single earthquake, recorded on a global network of seismometers. In practice, 1-D models of the Earth are typically constrained from data from hundreds of earthquakes, but you may be surprised how accurate of a model we can achieve with only one earthquake.

In principle, you could do this exercise using data from any earthquake large enough to be detected globally, but we will focus on a specific recent event that yielded very clear body wave arrivals. The event took place earlier this year, on January 9th, and was located in the Tanimbar Islands regions of Indonesia. The event was located at a depth of 105 km and an estimated moment magnitude of Mw 7.6. The IRIS event page is here: https://ds.iris.edu/ds/nodes/dmc/tools/event/11643012

As we have done previously, we will use Obspy's IRIS client to access the data. Downloading the seismic data will take a couple of minutes... It is grabbing 20 minutes of vertical component seismic data from >60 stations! Run the code block below to download the data.

In [ ]:
#imports
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from obspy.geodetics import gps2dist_azimuth #helper function for calculating distances and angles between points
from obspy.geodetics import kilometer2degrees #helper function for converting between kilometers and degrees on Earth
from obspy.geodetics import degrees2kilometers

#initialize IRIS client
client = Client("IRIS") 

#Indonesia event
origin = UTCDateTime('2023-01-09:17:47:35') #origin time of Mw 7.6 event in Tanimbar Islands, Indonesia
evlo = 130.0 #event longitude
evla = -7.0 #event latitude

#--- 
# For the interested: The parameters below are for the Mw 7.8 Turkey earthquake. You could use it for the analysis,
# but picking travel times is a bit messier. 
#---
#origin = UTCDateTime('2023-02-06:01:17:35') #origin time of Mw 7.8 event in Turkey
#evlo = 37.04
#evla = 37.16

starttime = origin
endtime = starttime + 60.*20 # we want 20 minutes of seismic data, starting at the origin time
st = client.get_waveforms("IU", "*", "00", "BHZ", starttime, endtime) #download waveforms
inv = client.get_stations(network='IU',starttime=starttime,endtime=endtime) #download station data to an 'inventory'

### Plotting a seismic 'record section'

To visualize seismic data from one earthquake recorded on many stations, seismologists often plot waveforms organized by epicentral distance. This type of plot is referred to as a *record section*. Most commonly, you will see distance on the x-axis (increasing to the right) and time on the y-axis (increasing up), so each seismogram is turned on its side compared to how you may be used to seeing them. 

Below, a function `plot_record_section` is provided, which takes several input arguments, including the obspy stream containing the waveform data that you just downloaded, and an obspy 'inventory' object, which contains information about the location of each seismometer (among other metadata), which was also downloaded. 

In [ ]:
def plot_record_section(st,inv,eq_longitude=evlo,eq_latitude=evla,scale_factor=5):
    '''
    inputs----------------------------------------
    
    st: obspy stream containing data to be plotted (downloaded w/ client.get_waveforms)
    inv: obspy inventory object containing station metadata (downloaded w/ client.get_stations)
    eq_longitude: longitude of earthquake (defaults to 'evlo', which is defined above)
    eq_latitude: latitude of earthquake (defaults to 'evla', which is defined above)
    scale_factor: scale factor for waveforms (defaults to 5, larger values will amplify seismograms more)
    '''
    fig = plt.figure(figsize=[7,9])
    
    #bandpass filter the stream-------------------------------------
    freqmin = 1./10.
    freqmax = 5.0
    stc = st.copy() #copy the stream
    stc.taper(0.05)     #taper (should always do before applying a filter)
    stc.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=2,zerophase=True)
    
    #loop through and plot each trace. First, we must find the epicentral distance!
    for tr in stc:
        station = inv.select(station=tr.stats.station)[0][0]
        dist_m,az,baz = gps2dist_azimuth(eq_latitude,eq_longitude,station.latitude,station.longitude)
        dist_km = dist_m/1000.
        dist_deg = kilometer2degrees(dist_km)
        
        if dist_deg > 100.0:
            continue
        
        #scale the data before plotting
        tr.normalize() # normalize the trace so the maximum value is 1
        tr.data *= scale_factor #scale the data so the maximum value is 'scale_factor'
        
        time = tr.times()
        plt.plot(tr.data+dist_deg,time,color='C0',alpha=0.5)
        
    plt.xlim([0,100])
    plt.ylim([0,1200])
    plt.xlabel('distance (degrees)')
    plt.ylabel('time (s)')
    plt.show()

###  <font color='red'>Question 1 </font> 
Make a record section of your data by calling the `plot_record_section` function.

In [ ]:
#Answer Q1 here:

### Picking P-wave travel times

In the plot above, you should hopefully see seismic waveforms ordered by increasing epicentral distance, and for the majority of the waveforms, you should see clear first arriving energy, which indicates the P-wave arrival. Since time = 0 corresponds to the earthquake origin time, the onset time of the P-wave at each epicentral distance corresponds to the travel time of a P-wave traveling that distance. Thus, by measuring P-wave travel times from the plot above, we can constrain the travel time curve $T(X)$.

###  <font color='red'>Question 2 </font> 
Pick P-wave travel times from the record section for epicentral distances of up to 94 degrees. To pick a P-wave travel time for a given distance, hover your mouse pointer near the P-wave first arrival, and read off the ordered pair that should show up in the lower right hand corner of the plot (x = distance, y = time). This is a very crude method of picking, which can lead to inaccuracies of up to several seconds, but for the purposes of this lab, that is acceptable. You do not need to make a pick on every single seismogram. The important part is that the travel time curve is relatively well sampled, which you should be able to do with a dozen or so picks. 

You can keep track of the picks however you like (e.g., spreadsheet, pen and paper, python list), but ultimately you will need to get the data into this notebook in python. When you are done with the picking, you should have two python lists that are the same length: one list containing the epicentral distances at which picks were made (in degrees), and one list containing the corresponding P-wave travel time picks themselves (in seconds). **Important: although there is no seismogram observed at 0 degrees distance, make sure to include the point 0,0 in your travel time curve. Thus, your lists should have travel times corresponding to distances ranging from 0 to 94 degrees.**

Note, typically when using matplotlib in Jupyter notebooks, figures will not be interactice, and you will not be able to see the coordinates under your mouse pointer. To enable that option in this lab, we used a different matplotlib "backend" called `%matplotlib notebook`, instead of `%matplotlib inline`. There are other differences between the backends, so you may notice matplotlib behaving strangely compared to how you are used to.

In [ ]:
#Answer Q2 here.

### Fitting a smooth curve to your travel time picks

Although we have made travel time picks at a set of discrete distances, we know that the travel time curve should be a smooth and continuous function of $X$. Additionally, due to inaccuracies in our picking, the picks we made contain some error about the 'true' travel time curve. To account for both of these factors, we will fit a smooth curve to the picks we made. 

In particular, the curve we want to fit to our travel time picks is a *4th order polynomial* (https://en.wikipedia.org/wiki/Quartic_function). To perform this fit, you could set it up as a linear inverse problem of the form $\mathbf{Gm} = \mathbf{d}$, and find the least squares solution, as you have done in previous labs. However, numpy provides us with a very simple function for polynomial curve fitting, called `numpy.polyfit`, which we can take advantage of.

`numpy.polyfit` takes in three arguments: i) a set of x-points (here would be the list of distances), ii) a set of y-points (here would be the list of travel times), and iii) the degree of the polynomial you would like to use (here should be 4), and returns a list of the coefficients of the polynomial **ordered with the coefficient corresponding to the highest power first**. In other words, if `numpy.polyfit` returned a list [1,2,3,4,5], the polynomial it is describing is

$f(x) = x^4 + 2x^3 + 3x^2 + 4x + 5$

The documentation for `numpy.polyfit` is here: https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html

###  <font color='red'>Question 3 </font> 

Fit a 4th order polynomial to your travel time picks using `numpy.polyfit`. Using the coefficients returned by `numpy.polyfit`, construct a smooth travel time curve T(X) for distances ranging from 0 to 94 degrees, and plot the results. 

Hint: Create a new numpy array of distances from 0 - 94 degrees using, for example `np.linspace`, and find the travel times at each distance based on the polynomial described by the results of `numpy.polyfit`. Your array of distances should have at least one point per degree. Also, you can plot the curve overlain on the record section if you don't create a new figure object, and instead simply call `plt.plot` in the cell below (i.e., the curve will be added to the record section plot above).


In [ ]:
#Answer Q3 here.


### Invert your P-wave travel times for the Earth's mantle structure

Now that you have a smooth T(x) curve for P-wave travel times, you are ready to create a 1-D model of $V_P$ in Earth's mantle. There are numerous different approaches that one might take to invert a travel time curve, but here we will use a classical approach based on the *Herglotz-Wiechert* formula: 

$
\displaystyle
\ln{\left(\frac{r_e}{r}\right)} = \frac{1}{\pi}  \int_0^{X_1} \cosh^{-1}\left(\frac{p}{u}\right)dX
$

where $r$ is radius, $r_e$ is the radius of the Earth, $p$ is the derivative of the travel time curve (i.e., $\frac{dT}{dX}$), and $u$ is the ray parameter (slowness) of a ray that has traveled a distance of $X_1$ degrees. Essentially, this formula uses the knowledge that the slope at a point of the $T(X)$ curve corresponds to 1/velocity at some depth (or radius) in the Earth's interior, and is used to solve for the radius $r$ at which a particular velocity occurs. Thus, to constrain the velocity-depth profile, the formula must be applied repeatedly for different values of $u$.

While this approach has several limiations (e.g., it breaks down in the presence of a low velocity zone), it was widely used in the era when seismologists were first beginning to construct body-wave travel time tables and to piece together the average seismic velocity and density structure of Earth's mantle (e.g., Bullen, 1956). For more details of the Herglotz-Wiechert formula, see Chapter 20 of *Foundations of Modern Global Seismology*. 

Note, the formula above is slightly different than what was presented in the lecture slides. This is because here we are using spherical coordinates instead of cartesian coordinates.

###  <font color='red'>Question 4 </font> 
Take the derivative of your smooth travel time curve to get $\frac{dT}{dX}$ (i.e., take the derivative of the curve that you created using a 4th order polynomial fit). To do this, we will use the `numpy.gradient` function. The problem is set up for you in the box below.

In [ ]:
#Answer Q4 here (i.e., complete what has been started)

#---------------------------------------------
#Take derivative of travel time curve (dT/dX)
#---------------------------------------------
P_times =           #fill in the name of the variable containing your travel times here
dist_degrees =      #fill in the name of the variable containing your distances here
dist_radians = np.deg2rad(dist_degrees) #use radians instead of degrees!
slowness_srad = np.gradient(P_times,dist_radians) #derivative of travel time curve

###  <font color='red'>Question 5 </font> 

Run the cell below to invert the $T(X)$ curve for P-wave velocity as a function of depth $V_P(z)$.

In [ ]:
from scipy.integrate import trapz

#---------------------------------------------------
# define function to do the numerical integration
#---------------------------------------------------
def herglotz_wiechert(p,u,x,x_lim):
    '''
    Uses the Herglotz-Weichert formula to find the depth z at which a slowness p (1/v) occurs
    
    p = derivative of travel time curve (ie. dT/dX)
    u = slowness of wave that has traveled a distance of x
    x = distance that the wave with slowness u has traveled
    x_lim = distance at which slowness u occurs
    '''
    x_ = np.linspace(0,x_lim,100)
    p_ = np.interp(x_,x,p)

    return (1./np.pi)*trapz(np.arccosh(p_/u),x_)

#---------------------------------------------------
# use the Herglotz-Wiechert formula to find the mantle velocity profile
#---------------------------------------------------

p = slowness_srad #dT/dX curve
x = dist_radians
r_e = 6371.0 #radius of the Earth (in km)

vel = [] #empty list to store velocities
depth = [] #empty list to store depths

#loop through slownesses in the dT/dX curve
for i in range(0,len(p)):
    
    u = p[i] #slowness at index i
    x_lim = x[i] #upper integration limit (i.e., distance that a ray with slowness u traveled)
    
    #call the herglotz_wiechert function
    ans = herglotz_wiechert(p=p,u=u,x=x,x_lim=x_lim)
    
    #solve the left hand side of the herglotz_weichert formula (ie. ln(r_e/r)) for r
    radius_here = r_e/np.exp(ans)
    depth_here = r_e - radius_here #convert radius r to depth
    depth.append(depth_here) #append depth to list

    #convert slowness in s/radian to s/km
    deg2km = degrees2kilometers(1,radius=radius_here)
    p_here_skm = np.deg2rad(u)/deg2km
    vel.append(1./p_here_skm) #append the velocity to list (i.e., 1/slowness)

###  <font color='red'>Question 6 </font> 

i) Plot the $V_P(z)$ curve you constructed from the Herglotz-Wiechert inversion. Label axes.

ii) On the course repository, you will find the file 'prem.nd', which contains data of the 'Preliminary Reference Earth Model' (aka PREM). Although this model was introduced over 40 years ago (Dziewonski and Anderson, 1981), it is still a commonly used model today. There are 6 columns in prem.nd, which correspond to

depth(km), Vp (km/s), Vs (km/s), density (g/cm^3), Qp (P-wave attenuation factor), Qs (S-wave attenuation factor)

Plot the Vp structure of PREM compared to your model. Where do you notice differences in the mantle velocities? What could account for these differences?

In [ ]:
#Answer Q6 here

#load the PREM model into a numpy array.
prem = np.loadtxt('../data/lab_04/prem.nd')

###  <font color='red'>Question 7 </font> 

The $V_P(z)$ curve you constructed has very large values of P-wave velocity (> 17 km/s) at depths below 2800 km (i.e., depths corresponding to the core). Should these values be interpretted? Why or why not?

In [ ]:
#Answer Q7 here.
